# Optimisers

単純な線形回帰を考えよう。いくつかのダミーデータを作り、損失を計算し、パラメータ`W`と`b`のために勾配を計算するためにバックプロパゲーションする。

In [18]:
using Flux
using Flux: Tracker, Params

W = param(rand(2,5))
b = param(rand(2))

predict(x) = W*x .+b
loss(x,y) = sum((predict(x) .-y).^2)

x,y = rand(5), rand(2)
@show l = loss(x,y) # ~3

@show Θ　= Params([W,b])
grads = Tracker.gradient(()->loss(x,y), Θ)

l = loss(x, y) = 3.387532384382774 (tracked)
Θ = Params([W, b]) = Params([Flux.Tracker.TrackedReal{Float64}[0.387905 (tracked), 0.721443 (tracked)], Flux.Tracker.TrackedReal{Float64}[0.905276 (tracked) 0.274859 (tracked) 0.425754 (tracked) 0.0999047 (tracked) 0.0495313 (tracked); 0.532982 (tracked) 0.111951 (tracked) 0.355829 (tracked) 0.486555 (tracked) 0.0532328 (tracked)]])


Grads(...)


損失を改善（削減）するため、勾配を使って、各パラメータを更新したい。  
これをするための一つの方法は次の通り。

In [19]:
using Flux.Tracker: grad, update!

function sgd()
    η = 0.1　# learning rate
    for p in (W,b)
        update!(p,-η*grads[0])
    end
end

sgd (generic function with 1 method)

`sgd`を呼ぶなら、パラメータ`W`と`b`は変化して損失は下がるはず。

ここには２つのピースがある。一つはモデルのための学習可能なパラメータのリスト（この場合は`[W, b]`）が必要なこと、そしてもう一つは更新ステップだ。  
この場合、更新はシンプルに勾配降下法（`x .= η .* Δ`）だが、モーメンタム（運動量）を追加するなどのもっと高度なことを選択するかもしれない。

この場合、編集を得るのは簡単だが、複雑なレイヤースタックがあると、より面倒になることが想像できる。

In [20]:
m = Chain(
    Dense(10,5,σ),
    Dense(5,2),
    softmax)

Chain(Dense(10, 5, NNlib.σ), Dense(5, 2), NNlib.softmax)

`[m[1].W, m[1].b, ...]`と書く代わりに、Fluxは君のために`params(m)`でモデルの中のすべてのパラメータのリストを返すparams関数を提供する。

アップデートステップでは、上述のループを書いても心配は無い。これはうまくいく。  
しかしFluxはそれをより便利にする様々なオプティマイザを提供する。

In [22]:
opt = SGD([W,b], 0.1) # 勾配降下法と学習率

opt() # アップデートを実行して、Wとbを修正する

オプティマイザはパラメータリストを取り、上述の`update`と同じことをする関数を返す。  
`opt`か`update`をトレーニングループに渡すことができ、データの各ミニバッチの後にオプティマイザを実行するだろう。

In [24]:
#! paramsを使った場合はこう書けそう...
opt2 = SGD(params(m), 0.1)
opt2() 

## Optimiser Reference
---
すべてのOptimisersは、呼ばれたときに、渡されたパラメータを更新する関数を返す。

[WIP]